In [19]:
import os
import pandas as pd

BASE_PATH = "/Users/jesus/Desktop/4geeks/4geeks - booster random forest/jesus-cuenca-booster-random-forest-machine-learning-python-template/data/processed/excels"

X_train = pd.read_excel(f"{BASE_PATH}/X_train_sel_k4.xlsx")
X_test = pd.read_excel(f"{BASE_PATH}/X_test_sel_k4.xlsx")
y_train = pd.read_excel(f"{BASE_PATH}/y_train.xlsx")
y_test = pd.read_excel(f"{BASE_PATH}/y_test.xlsx")

In [20]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state = 42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [21]:
from xgboost import XGBClassifier

model = XGBClassifier(
    # 🌱 CONTROL DE CRECIMIENTO DEL ÁRBOL
    max_depth=8,                # Se reduce la profundidad para evitar sobreajuste
    min_child_weight=2,         # Mayor poda de nodos irrelevantes
    gamma=0.3,                  # Se aumenta la poda para evitar ramas débiles
    
    # 🍃 REGULARIZACIÓN (Evitar Overfitting)
    subsample=0.75,             # Menos datos por árbol para mejorar generalización
    colsample_bytree=0.7,       # Menos features por árbol para más diversidad
    colsample_bylevel=0.7,      
    colsample_bynode=0.7,       
    lambda_=2,                  # Regularización L2 más fuerte
    alpha=1,                    # Regularización L1 más agresiva
    
    # 🔥 BOOSTING CONTROL (Evitar Overfitting)
    learning_rate=0.03,         # Menor tasa de aprendizaje para mejor estabilidad
    n_estimators=1000,          # Más árboles, pero con early stopping para detener cuando mejore     
    objective="multi:softmax",  # Clasificación multiclase
    num_class=3,                # Número de clases en el target
    
    # 📈 CONTROL DEL GRADIENTE
    booster="gbtree",           
    tree_method="hist",         
    scale_pos_weight=1,         
    
    # ⚡ OPTIMIZACIÓN DEL PROCESO
    grow_policy="lossguide",    # Mejor división basada en ganancia en lugar de profundidad
    importance_type="gain",     
    verbosity=1,                
    
    # 🏎️ PARALELIZACIÓN Y VELOCIDAD
    n_jobs=-1,                  
    random_state=42             

)
model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [17:24:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "lambda_", "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(alpha=1, base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=0.7, colsample_bynode=0.7, colsample_bytree=0.7,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=0.3, grow_policy='lossguide', importance_type='gain',
              interaction_constraints=None, lambda_=2, learning_rate=0.03,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, ...)

In [22]:
import numpy as np

X_test = np.array(X_test)  # Si X_test es un DataFrame, conviértelo a array
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [23]:
from sklearn.metrics import accuracy_score

# Predicciones en el set de entrenamiento
y_train_pred = model.predict(X_train)

# Predicciones en el set de prueba
y_test_pred = model.predict(X_test)

# Métricas de precisión
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Precisión en entrenamiento: {train_accuracy:.8f}")
print(f"Precisión en prueba: {test_accuracy:.8f}")



Precisión en entrenamiento: 0.87133550
Precisión en prueba: 0.75974026


In [24]:
model.save_model("/Users/jesus/Desktop/4geeks/4geeks - árbol de decisiones/booster-random-forest-machine-learning-python-template/data/processed/diccionarios/xgb_classifier_default_42.json")